## Prepare Training Data

In [1]:
import json
import torchaudio
import matplotlib.pyplot as plt
import torch
from datetime import time
import numpy as np

/Users/danilfedorovsky/miniforge3/envs/pytorch/lib/python3.9/site-packages/torchaudio/_internal/module_utils.py:99: UserWarning: Failed to import soundfile. 'soundfile' backend is not available.
  warnings.warn("Failed to import soundfile. 'soundfile' backend is not available.")


In [2]:
## decide what to load
SESSION = "S02"
transcript_path = "/Users/danilfedorovsky/Documents/10 Collection/00 Studium/00 Letztes Semester/Masterarbeit/Data/CHiME5/transcriptions/dev/" + SESSION + ".json"
#'/project/data_asr/CHiME5/data/CHiME5/transcriptions/dev/' + SESSION
data_path = "/Users/danilfedorovsky/Documents/10 Collection/00 Studium/00 Letztes Semester/Masterarbeit/Data/CHiME5/audio/dev/"

In [3]:
with open(transcript_path, 'r') as f:
    transcript = json.load(f)

In [4]:
u1_times = []
speaker_times = []
for entry in transcript:
    #print(entry["start_time"]["U01"])
    #print(entry["end_time"]["U01"])
    try:
        speaker_times.append([entry["speaker"],time.fromisoformat("0"+str(entry["start_time"]["U01"])+"0"),time.fromisoformat("0"+str(entry["end_time"]["U01"])+"0")])
        u1_times.append([time.fromisoformat("0"+str(entry["start_time"]["U01"])+"0"),time.fromisoformat("0"+str(entry["end_time"]["U01"])+"0")])
    except:
        # if redacted
        pass

print(speaker_times[0])
print(u1_times[0])
print(len(speaker_times)==len(u1_times))

['P05', datetime.time(0, 0, 40, 630000), datetime.time(0, 0, 43, 850000)]
[datetime.time(0, 0, 40, 630000), datetime.time(0, 0, 43, 850000)]
True


In [5]:
# 2h 28 min 24 s = 8904s
P05, sample_rate = torchaudio.load(data_path+"S02_P05.wav")
P06, _ = torchaudio.load(data_path+"S02_P06.wav")
P07, _ = torchaudio.load(data_path+"S02_P07.wav")
P08, _ = torchaudio.load(data_path+"S02_P08.wav")
U01, _ = torchaudio.load(data_path+"S02_U01.CH1.wav")
U012, _ = torchaudio.load(data_path+"S02_U01.CH2.wav")
#P05 = P05[0].reshape(1,-1)
#P06 = P06[0].reshape(1,-1)
#P07 = P07[0].reshape(1,-1)
#P08 = P08[0].reshape(1,-1)
print(U01.size(dim=1)/sample_rate, "seconds")

8904.04 seconds


In [6]:
# Cut the relevant segments
X = []#np.array([])#[]#torch.Tensor()
Y = []#np.array([])#[]#torch.Tensor()

def time_to_sample(time:time):
    return int(sample_rate*(time.hour*3600+time.minute*60+time.second+time.microsecond/1000000))

# Input
for start, end in u1_times:
    start = time_to_sample(start)
    end = time_to_sample(end)
    #X = torch.cat((X, U01[0][start:end]), 0)
    X.append([U01[0][start:end],U012[0][start:end]])
print("Input ready")

# Label
for speaker,start,end in speaker_times:
    #print(speaker,start,end)
    start = time_to_sample(start)
    end = time_to_sample(end)
    if speaker == "P05":
        #Y = torch.cat((Y, P05[0][start:end]), 0)
        #Y.append([P05[0][start:end],P05[1][start:end]])
        Y.append(P05[0][start:end])
    if speaker == "P06":
        #Y = torch.cat((Y, P06[0][start:end]), 0)
        Y.append(P06[0][start:end])
    if speaker == "P07":
        #Y = torch.cat((Y, P07[0][start:end]), 0)
        Y.append(P07[0][start:end])
    if speaker == "P08":
        #Y = torch.cat((Y, P08[0][start:end]), 0)
        Y.append(P08[0][start:end])

print("Label ready")
#print(X.size(0)==Y.size(0))
print(len(X)==len(Y))

Input ready
Label ready
True


## Training on CHIME

In [29]:
from torch.nn import Module, LeakyReLU, MaxPool1d, Conv2d, Linear, Softmax, BatchNorm1d, CrossEntropyLoss, BCELoss, Sigmoid, Tanh, LSTM, MSELoss
from torch import flatten
from torch.optim import Adam
import time
import torch
from pytorch_model_summary import summary
from tqdm import tqdm
import torchaudio

In [30]:
HIDDEN_SIZE=8
INIT_LR = 0.01
BATCH_SIZE = 1#64
SAMPLE_RATE = 16000
EPOCHS = 1
INPUT_CHANNEL = 2
REFERENCE_CHANNEL = 0

N_FFT = 1024
N_HOP = 256
stft = torchaudio.transforms.Spectrogram(
    n_fft=N_FFT,
    hop_length=N_HOP,
    power=None,
)
istft = torchaudio.transforms.InverseSpectrogram(n_fft=N_FFT, hop_length=N_HOP)
global i
i = 0

def prep_x(trainX,i):
        return stft((torch.cat((trainX[i][0].unsqueeze(1),trainX[i][1].unsqueeze(1)),1)).reshape(2,-1)).type(torch.complex64)

In [31]:
# # MASK NET

class MaskNet(Module):
    def __init__(self,noise=False):
        super(MaskNet, self).__init__()
        # First subnet for speech prediction
        self.type(torch.cfloat)
        self.noise = noise
        self.lstm = LSTM(input_size=INPUT_CHANNEL, hidden_size=HIDDEN_SIZE, num_layers=2, bidirectional=True)
        self.fc = Linear(in_features=HIDDEN_SIZE*2 ,out_features=1)
        self.sigmoid = Sigmoid()
        # Second subnet for noise prediction
        self.noise2 = noise
        self.lstm2 = LSTM(input_size=INPUT_CHANNEL, hidden_size=HIDDEN_SIZE, num_layers=2, bidirectional=True)
        self.fc2 = Linear(in_features=HIDDEN_SIZE*2 ,out_features=1)
        self.sigmoid2 = Sigmoid()

    def forward(self,x):
        # Speech prediction
        y = x.reshape(513,-1,INPUT_CHANNEL)#(513,251,INPUT_CHANNEL)
        y, (h_n, c_n) = self.lstm(y)
        y = self.fc(y).type(torch.double)
        speech_pred = self.sigmoid(y)
        speech_pred = speech_pred.reshape(513,-1).type(torch.cfloat)
        #print("speech_pred.shape",speech_pred.shape)

        # Noise prediction
        z = x.reshape(513,-1,INPUT_CHANNEL)
        z, (h_n, c_n) = self.lstm2(z)
        z = self.fc2(z).type(torch.double)
        noise_pred = self.sigmoid2(z)
        noise_pred = noise_pred.reshape(513,-1).type(torch.cfloat)

        psd_transform = torchaudio.transforms.PSD()
        psd_speech = psd_transform(x, speech_pred)
        psd_noise = psd_transform(x, noise_pred)
        mvdr_transform = torchaudio.transforms.SoudenMVDR()
        x = x.type(torch.cfloat)
        stft_souden = mvdr_transform(x, psd_speech, psd_noise, reference_channel=REFERENCE_CHANNEL)
        #print("stft_souden",stft_souden.shape)
        waveform_souden = istft(stft_souden, length=len(X[i][0]))#X[i].shape[-1])
        waveform_souden[waveform_souden != waveform_souden] = 1.0000e-10 
        print(waveform_souden)
        return waveform_souden.reshape(-1)

print(summary(MaskNet(),torch.zeros((INPUT_CHANNEL, 513, 251))))


tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<AsStridedBackward0>)
------------------------------------------------------------------------------------------------
      Layer (type)                                 Output Shape         Param #     Tr. Param #
            LSTM-1     [513, 251, 16], [4, 251, 8], [4, 251, 8]           2,432           2,432
          Linear-2                                [513, 251, 1]              17              17
         Sigmoid-3                                [513, 251, 1]               0               0
            LSTM-4     [513, 251, 16], [4, 251, 8], [4, 251, 8]           2,432           2,432
          Linear-5                                [513, 251, 1]              17              17
         Sigmoid-6                                [513, 251, 1]               0               0
Total params: 4,898
Trainable params: 4,898
Non-trainable params: 0
----------------------------------------------------------------------------------------------

In [32]:

#device =  torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = MaskNet()#.to(device)
model.type(torch.complex64)

opt = Adam(model.parameters(), lr=INIT_LR)

lossMSE = MSELoss()

H = {
    "train_loss":[],
    "train_acc":[],
    "val_loss":[],
    "val_acc":[]
}

print("[INFO] training the network...")
#startTime = time.time()

for epoch in range(0, EPOCHS):
    print("Epoch:",epoch,"/1")
    # Train Mode
    model.train()
    
    # Initialize
    totalTrainLoss = 0
    totalValLoss = 0
    trainCorrect = 0
    valCorrect = 0

    trainX = X
    trainY = Y

    global i
    for i in tqdm(range(0,100)):#len(trainX))):
        (x, y) = (stft((torch.cat((trainX[i][0].unsqueeze(1),trainX[i][1].unsqueeze(1)),1)).reshape(2,-1)),trainY[i])#(trainX[i],trainY[i])
        #(x, y) = (x.to(device), y.to(device))
        # perform a forward pass and calculate the training loss
        #print(next(model.parameters()).is_cuda)
        print(x.shape)
        pred = model(x)
        loss = lossMSE(pred, y)

        #print("Loss:",loss)
        # zero out the gradients, perform the backpropagation step,
        # and update the weights
        opt.zero_grad()
        loss.backward()
        opt.step()
        # add the loss to the total training loss so far and
        # calculate the number of correct predictions
        totalTrainLoss += loss
        # trainCorrect += (pred.argmax(1) == y).type(
        #     torch.float).sum().item()

[INFO] training the network...
Epoch: 0 /1


  0%|          | 0/100 [00:00<?, ?it/s]

torch.Size([2, 513, 202])
tensor([ 0.0005,  0.0031,  0.0023,  ..., -0.0079, -0.0086, -0.0067],
       grad_fn=<AsStridedBackward0>)


  1%|          | 1/100 [00:02<04:11,  2.54s/it]

torch.Size([2, 513, 179])
tensor([ 0.0077,  0.0080,  0.0104,  ..., -0.0022, -0.0011,  0.0081],
       grad_fn=<AsStridedBackward0>)


...

 98%|█████████▊| 98/100 [05:50<00:08,  4.19s/it]

torch.Size([2, 513, 83])
tensor([1.0000e-10, 1.0000e-10, 1.0000e-10,  ..., 1.0000e-10, 1.0000e-10,
        1.0000e-10], grad_fn=<AsStridedBackward0>)


 99%|█████████▉| 99/100 [05:51<00:03,  3.30s/it]

torch.Size([2, 513, 839])
tensor([1.0000e-10, 1.0000e-10, 1.0000e-10,  ..., 1.0000e-10, 1.0000e-10,
        1.0000e-10], grad_fn=<AsStridedBackward0>)


100%|██████████| 100/100 [05:59<00:00,  3.59s/it]


In [33]:
PATH = "./modelsave.pt"
torch.save(model.state_dict(), "./modelsave.pt")

## Test model

In [34]:
PATH = "./modelsave.pt"
model = MaskNet()
model.load_state_dict(torch.load(PATH))
model.eval()

MaskNet(
  (lstm): LSTM(2, 8, num_layers=2, bidirectional=True)
  (fc): Linear(in_features=16, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (lstm2): LSTM(2, 8, num_layers=2, bidirectional=True)
  (fc2): Linear(in_features=16, out_features=1, bias=True)
  (sigmoid2): Sigmoid()
)

In [36]:
j = 120
#pred_null = model(stft((torch.cat((trainX[j][0].unsqueeze(1),trainX[j][1].unsqueeze(1)),1)).reshape(2,-1)).type(torch.complex64))

In [39]:
def save_sample(i):
    pred_null = model(stft((torch.cat((trainX[i][0].unsqueeze(1),trainX[i][1].unsqueeze(1)),1)).reshape(2,-1)).type(torch.float))
    torchaudio.save("sample_model_output.wav",pred_null.reshape(1,-1),sample_rate)
    torchaudio.save("sample_reference.wav", trainY[i].unsqueeze(1).reshape(1,-1),sample_rate)
    torchaudio.save("sample_input.wav", trainX[i][0].unsqueeze(1).reshape(1,-1),sample_rate)

In [40]:
save_sample(50)

tensor([1.0000e-10, 1.0000e-10, 1.0000e-10,  ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00], grad_fn=<AsStridedBackward0>)


## Librispeech Dataset + Noise

In [2]:
import os
import torchaudio
import random
import torch

N_PATH = "/Users/danilfedorovsky/Documents/10 Collection/00 Studium/00 Letztes Semester/Masterarbeit/Data/noise/free-sound/"
S_PATH = "/Users/danilfedorovsky/Documents/10 Collection/00 Studium/00 Letztes Semester/Masterarbeit/Data/LibriSpeech/dev-clean/"
def load_noise(N_PATH=N_PATH):
    noise = []
    for file in  os.listdir(N_PATH):
        if file[-4:] == ".wav":
            sound, _ = torchaudio.load(N_PATH+file)
            noise.append(sound)
    return noise


def load_speech(S_PATH=S_PATH):
    speech = []
    for folder in  os.listdir(S_PATH):
        if os.path.isdir(S_PATH+folder):
            for subfolder in os.listdir(S_PATH+folder):
                if os.path.isdir(S_PATH+folder+"/"+subfolder):
                    for file in os.listdir(S_PATH+folder+"/"+subfolder):
                        if file[-5:] == ".flac":
                            sound, _ = torchaudio.load(S_PATH+folder+"/"+subfolder+"/"+file)
                            speech.append(sound)
    return speech

/Users/danilfedorovsky/miniforge3/envs/pytorch/lib/python3.9/site-packages/torchaudio/_internal/module_utils.py:99: UserWarning: Failed to import soundfile. 'soundfile' backend is not available.
  warnings.warn("Failed to import soundfile. 'soundfile' backend is not available.")


In [3]:
speech = load_speech()
noise = load_noise()
print(len(speech))
print(len(noise))

2703
843


In [4]:
def add_noise_to_speech(speech, noise):
    X = []
    for sample in speech:
        len_noise = 0
        len_speech = 1
        sample_noise = torch.Tensor([])
        while len_noise<len_speech:
            sample_noise = random.choice(noise)
            len_speech = sample.shape[1]
            len_noise = sample_noise.shape[1]
        sample_noise = torch.narrow(sample_noise,1,0,len_speech)
        x = torch.add(sample,sample_noise*0.2)
        X.append(x)
    return X

In [5]:
X = add_noise_to_speech(speech,noise)
Y = speech

## Training on LibreSpeech

In [6]:
from torch.nn import Module, LeakyReLU, MaxPool1d, Conv2d, Linear, Softmax, BatchNorm1d, CrossEntropyLoss, BCELoss, Sigmoid, Tanh, LSTM, MSELoss
from torch import flatten
from torch.optim import Adam, RMSprop, SGD
import time
import torch
from pytorch_model_summary import summary
from tqdm import tqdm
import torchaudio

In [7]:
HIDDEN_SIZE=64
INIT_LR = 0.01
BATCH_SIZE = 1#64
SAMPLE_RATE = 16000
EPOCHS = 1
INPUT_CHANNEL = 1
REFERENCE_CHANNEL = 0

N_FFT = 1024
N_HOP = 256
stft = torchaudio.transforms.Spectrogram(
    n_fft=N_FFT,
    hop_length=N_HOP,
    power=None,
)
istft = torchaudio.transforms.InverseSpectrogram(n_fft=N_FFT, hop_length=N_HOP)
global i
i = 0

def prep_x(trainX,i):
        return stft((torch.cat((trainX[i][0].unsqueeze(1),trainX[i][1].unsqueeze(1)),1)).reshape(2,-1)).type(torch.complex64)

In [8]:
# # MASK NET

class MaskNet(Module):
    def __init__(self,noise=False):
        super(MaskNet, self).__init__()
        # First subnet for speech prediction
        self.type(torch.cfloat)
        self.noise = noise
        self.lstm = LSTM(input_size=INPUT_CHANNEL, hidden_size=HIDDEN_SIZE, num_layers=2, bidirectional=True)
        self.fc = Linear(in_features=HIDDEN_SIZE*2 ,out_features=1)
        self.sigmoid = Sigmoid()
        # Second subnet for noise prediction
        self.noise2 = noise
        self.lstm2 = LSTM(input_size=INPUT_CHANNEL, hidden_size=HIDDEN_SIZE, num_layers=2, bidirectional=True)
        self.fc2 = Linear(in_features=HIDDEN_SIZE*2 ,out_features=1)
        self.sigmoid2 = Sigmoid()

    def forward(self,x):
        # Speech prediction
        y = x.reshape(513,-1,INPUT_CHANNEL)#(513,251,INPUT_CHANNEL)
        y, (h_n, c_n) = self.lstm(y)
        y = self.fc(y).type(torch.double)
        speech_pred = self.sigmoid(y)
        speech_pred = speech_pred.reshape(513,-1).type(torch.cfloat)
        #print("speech_pred.shape",speech_pred.shape)

        # Noise prediction
        z = x.reshape(513,-1,INPUT_CHANNEL)
        z, (h_n, c_n) = self.lstm2(z)
        z = self.fc2(z).type(torch.double)
        noise_pred = self.sigmoid2(z)
        noise_pred = noise_pred.reshape(513,-1).type(torch.cfloat)

        psd_transform = torchaudio.transforms.PSD()
        psd_speech = psd_transform(x, speech_pred)
        psd_noise = psd_transform(x, noise_pred)
        mvdr_transform = torchaudio.transforms.SoudenMVDR()
        x = x.type(torch.cfloat)
        stft_souden = mvdr_transform(x, psd_speech, psd_noise, reference_channel=REFERENCE_CHANNEL)
        #print("stft_souden",stft_souden.shape)
        waveform_souden = istft(stft_souden, length=len(X[i][0]))#X[i].shape[-1])
        waveform_souden[waveform_souden != waveform_souden] = 1.0000e-10 
        return waveform_souden.reshape(-1)

print(summary(MaskNet(),torch.zeros((INPUT_CHANNEL, 513, 251))))


/Users/danilfedorovsky/miniforge3/envs/pytorch/lib/python3.9/site-packages/torchaudio/functional/functional.py:190: UserWarning: The length of signal is shorter than the length parameter. Result is being padded with zeros in the tail. Please check your center and hop_length settings. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/SpectralOps.cpp:1111.)
  waveform = torch.istft(


---------------------------------------------------------------------------------------------------
      Layer (type)                                    Output Shape         Param #     Tr. Param #
            LSTM-1     [513, 251, 128], [4, 251, 64], [4, 251, 64]         133,632         133,632
          Linear-2                                   [513, 251, 1]             129             129
         Sigmoid-3                                   [513, 251, 1]               0               0
            LSTM-4     [513, 251, 128], [4, 251, 64], [4, 251, 64]         133,632         133,632
          Linear-5                                   [513, 251, 1]             129             129
         Sigmoid-6                                   [513, 251, 1]               0               0
Total params: 267,522
Trainable params: 267,522
Non-trainable params: 0
---------------------------------------------------------------------------------------------------


In [9]:

#device =  torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = MaskNet()#.to(device)
model.type(torch.complex64)

opt = SGD(model.parameters(),lr=INIT_LR)#RMSprop(model.parameters(), lr=INIT_LR)#Adam(model.parameters(), lr=INIT_LR)

lossMSE = MSELoss()

H = {
    "train_loss":[],
    "train_acc":[],
    "val_loss":[],
    "val_acc":[]
}

print("[INFO] training the network...")
#startTime = time.time()

for epoch in range(0, EPOCHS):
    print("Epoch:",epoch,"/1")
    # Train Mode
    model.train()
    
    # Initialize
    totalTrainLoss = 0
    totalValLoss = 0
    trainCorrect = 0
    valCorrect = 0

    trainX = X
    trainY = Y

    global i
    for i in tqdm(range(0,50)):#len(trainX))):
        (x, y) = (stft(trainX[i]),trainY[i].reshape(-1))#(trainX[i],trainY[i])
        #(x, y) = (x.to(device), y.to(device))
        # perform a forward pass and calculate the training loss
        #print(next(model.parameters()).is_cuda)
        pred = model(x)
        print(pred)
        loss = lossMSE(pred, y)

        #print("Loss:",loss)
        # zero out the gradients, perform the backpropagation step,
        # and update the weights
        opt.zero_grad()
        loss.backward()
        opt.step()
        # add the loss to the total training loss so far and
        # calculate the number of correct predictions
        totalTrainLoss += loss
        # trainCorrect += (pred.argmax(1) == y).type(
        #     torch.float).sum().item()

[INFO] training the network...
Epoch: 0 /1


  0%|          | 0/50 [00:00<?, ?it/s]/var/folders/wr/spkz8f691xl_gwp76c19_yv40000gn/T/ipykernel_75399/418719489.py:22: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Copy.cpp:250.)
  y = self.fc(y).type(torch.double)


tensor([ 3.4561e-12, -9.0862e-13, -5.5049e-13,  ..., -5.5481e-03,
        -4.7363e-03, -4.1809e-03], grad_fn=<ReshapeAliasBackward0>)


  2%|▏         | 1/50 [00:42<34:54, 42.75s/it]

tensor([-0.0074, -0.0123, -0.0107,  ...,  0.0252,  0.0083, -0.0196],
       grad_fn=<ReshapeAliasBackward0>)


...

 96%|█████████▌| 48/50 [1:09:02<02:28, 74.26s/it]

tensor([-0.0005, -0.0003, -0.0003,  ..., -0.0006, -0.0008, -0.0007],
       grad_fn=<ReshapeAliasBackward0>)


 98%|█████████▊| 49/50 [1:14:22<02:28, 148.02s/it]

tensor([-0.0015, -0.0016, -0.0017,  ..., -0.0002, -0.0005, -0.0009],
       grad_fn=<ReshapeAliasBackward0>)


100%|██████████| 50/50 [1:14:58<00:00, 89.96s/it] 


In [10]:
PATH = "./modelsaveLib.pt"
torch.save(model.state_dict(), PATH)

In [38]:
def save_sample(i,sample_rate=16000):
    pred_null = model(stft(trainX[i]))
    torchaudio.save("sample_model_output.wav",pred_null.reshape(1,-1),sample_rate)
    torchaudio.save("sample_reference.wav", trainY[i].reshape(1,-1),sample_rate)
    torchaudio.save("sample_input.wav", trainX[i].reshape(1,-1),sample_rate)

In [39]:
save_sample(10)